<a href="https://colab.research.google.com/github/basleal22/baslealwork/blob/main/NlP_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import zipfile
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

--2022-11-30 10:43:43--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.62.128, 142.250.188.48, 172.253.115.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.62.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2022-11-30 10:43:43 (78.4 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [2]:
zip_re= zipfile.ZipFile("nlp_getting_started.zip","r")
zip_re.extractall()
zip_re.close()

# becoming one with the data

In [3]:
import pandas as pd
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


# shuffle dataframe


In [4]:
train_df_shuffle=train_df.sample(frac=1, random_state=42)
train_df_shuffle.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


# Split data into train and val set

In [5]:
from sklearn.model_selection import train_test_split
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffle["text"].to_numpy(),
                                                                            train_df_shuffle["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)


In [6]:
len(train_sentences), len(val_sentences),len(train_labels),len(val_labels),len(train_df_shuffle)

(6851, 762, 6851, 762, 7613)

# converting text to numbers

In [7]:
#tokenization
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
 
text_vectorizer=TextVectorization(max_tokens=None,
                                  standardize="lower_and_strip_punctuation",
                                  split="whitespace",
                                  ngrams=None,
                                  output_mode="int",
                                  output_sequence_length=None)
                                  #pad_to_max_tokens=True)

In [8]:
  max([len(i.split())for i in train_sentences])

31

In [9]:
max_vocab_size=10000
max_word_len=15 
text_vectorizer=TextVectorization(max_tokens=max_vocab_size,
                                  output_mode='int',
                                  output_sequence_length=max_word_len)

In [10]:
#fit the text data to the train data
text_vectorizer.adapt(train_sentences)

In [11]:
sample_sent="there was a flood"
text_vectorizer([sample_sent])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,  23,   3, 232,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [12]:
import random

random_sent=random.choice(train_sentences)
print(random_sent)
text_vectorizer(random_sent)

Are the safety improvements being made to Route 198 in Buffalo doing more harm than good? http://t.co/rqlPSAYE3B


<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([  22,    2,  992,    1,  121,  299,    5, 1369,    1,    4, 2533,
        487,   51,  396,   76])>

In [13]:
words_in_vocab=text_vectorizer.get_vocabulary()
words_in_vocab[:10]
words_in_vocab[-10:]

['painthey',
 'painful',
 'paine',
 'paging',
 'pageshi',
 'pages',
 'paeds',
 'pads',
 'padres',
 'paddytomlinson1']

# creating an embedding

In [14]:
from tensorflow.keras import layers
embedding=layers.Embedding(input_dim=max_vocab_size,
                           output_dim=128,
                           input_length=max_word_len)

In [15]:
random_sent= random.choice(train_sentences)
print(random_sent)
# To embed a sentence we must convert the chosen statement to a positive number form
sample_embed=embedding(text_vectorizer([random_sent]))
sample_embed

Your Router is One of the Latest DDoS Attack Weapons http://t.co/vXxMvgtzvg #phone #gaming #tv #news


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03300295, -0.01377146,  0.01889059, ..., -0.02322875,
          0.01544168,  0.01292369],
        [-0.02867696, -0.01298251, -0.02189149, ..., -0.02280141,
         -0.02056426, -0.03989912],
        [ 0.01299408, -0.04858178,  0.04328047, ...,  0.03760214,
          0.01461351,  0.04559258],
        ...,
        [-0.01431976, -0.04889947,  0.00891112, ...,  0.00084781,
          0.04235318, -0.04702269],
        [-0.0409492 , -0.03260048, -0.00547798, ...,  0.02757546,
          0.02135488,  0.0308074 ],
        [-0.0245872 , -0.00877522,  0.01736826, ..., -0.0167435 ,
          0.04248554, -0.03606246]]], dtype=float32)>

#modeling a text dataset(running a series of experiments)
- baseline
- feed-forward neural network
- LSTM model
- GRU
- Bidirectional-LSTM
- 1D convolutional model
- Tensorflow hub pretrained Feature Extractor

In [16]:
#simple dense model
#create a callback model
import datetime
def create_tensorboard_callback(dir_name,experiment_name):
  log_dir=dir_name+'/'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f'saving tenorboard log files to: {log_dir}')
  return tensorboard_callback
SAVE_DIR='model_logs'

In [17]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string")
x= text_vectorizer(inputs)
x=embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_1=tf.keras.Model(inputs,outputs,name="model_1_dense")

In [18]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [19]:
model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])
history=model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

saving tenorboard log files to: model_logs/20221130-104347
Epoch 1/5
215/215 [==============================] - 12s 50ms/step - loss: 0.6148 - accuracy: 0.6895 - val_loss: 0.5401 - val_accuracy: 0.7546
Epoch 2/5
215/215 [==============================] - 5s 23ms/step - loss: 0.4446 - accuracy: 0.8170 - val_loss: 0.4716 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 5s 23ms/step - loss: 0.3486 - accuracy: 0.8589 - val_loss: 0.4602 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 5s 21ms/step - loss: 0.2857 - accuracy: 0.8889 - val_loss: 0.4679 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 5s 24ms/step - loss: 0.2387 - accuracy: 0.9098 - val_loss: 0.4785 - val_accuracy: 0.7861


In [20]:
model_1_predict=model_1.predict(val_sentences)
model_1_predict[:10]

24/24 [==============================] - 0s 3ms/step


array([[0.33538356],
       [0.7499366 ],
       [0.9979779 ],
       [0.1435603 ],
       [0.10656918],
       [0.9447408 ],
       [0.88775676],
       [0.99290717],
       [0.96595293],
       [0.31919366]], dtype=float32)

#now to compare to the labels we need to remove the 2nd dimension

In [21]:
model_1_predict=tf.squeeze(tf.round(model_1_predict))
model_1_predict[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

#visualizing learned embedding

In [22]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [23]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [24]:
import io

# # Create output writers
out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# # Write embedding vectors and words to file
for num, word in enumerate(words_in_vocab):
    if num == 0: 
      continue # skip padding token
    vec = embed_weights[num]
    out_m.write(word + "\n") # write words to file
    out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
out_v.close()
out_m.close()

In [25]:
try:
   from google.colab import files
except ImportError:
  pass
else:
  files.download("embedding_vectors.tsv")
  files.download("embedding_metadata.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#RNNs

In [26]:
#create an lstm model
from tensorflow.keras import layers
inputs= layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.LSTM(64,return_sequences=True)(x)#sets the sequences as they are and not deduct or squeeze any layer
x=layers.LSTM(64,activation="relu")(x)
x=layers.Dense(64,activation="sigmoid")(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_2=tf.keras.Model(inputs,outputs,name="model_2_lstm")


In [27]:
model_2.summary()

Model: "model_2_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                      

In [28]:
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
history=model_2.fit(train_sentences,train_labels, epochs=5, validation_data=(val_sentences,val_labels))

Epoch 1/5
215/215 [==============================] - 20s 70ms/step - loss: 0.2448 - accuracy: 0.8943 - val_loss: 0.5568 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 9s 40ms/step - loss: 0.1666 - accuracy: 0.9399 - val_loss: 0.6264 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 7s 31ms/step - loss: 0.1346 - accuracy: 0.9517 - val_loss: 0.6614 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 9s 40ms/step - loss: 0.1152 - accuracy: 0.9583 - val_loss: 0.7315 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 11s 52ms/step - loss: 0.1001 - accuracy: 0.9644 - val_loss: 0.7875 - val_accuracy: 0.7703


In [29]:
model_2_pred=model_2.predict(val_sentences)
model_2_pred[:10]

24/24 [==============================] - 1s 6ms/step


array([[0.04089222],
       [0.64268583],
       [0.99649715],
       [0.01742255],
       [0.00301979],
       [0.9764613 ],
       [0.6536528 ],
       [0.9966105 ],
       [0.996378  ],
       [0.46427408]], dtype=float32)

In [30]:
model_2_predict=tf.squeeze(tf.round(model_2_pred))
model_2_predict[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

# Model 3-GRU

In [31]:
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x= layers.GRU(64,return_sequences=True)(x)
x=layers.GRU(32,activation="relu",return_sequences=True)(x)
x=layers.GlobalAveragePooling1D()(x)
x=layers.Dense(64,activation="relu")(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_3=tf.keras.Model(inputs,outputs,name="model_3_gru")

In [32]:
model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])
history=model_3.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels))

Epoch 1/5
215/215 [==============================] - 11s 32ms/step - loss: 0.1733 - accuracy: 0.9498 - val_loss: 0.6900 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 9s 42ms/step - loss: 0.0978 - accuracy: 0.9628 - val_loss: 0.8087 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 9s 41ms/step - loss: 0.0839 - accuracy: 0.9660 - val_loss: 0.8862 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 8s 35ms/step - loss: 0.0760 - accuracy: 0.9714 - val_loss: 0.8497 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 10s 44ms/step - loss: 0.0669 - accuracy: 0.9750 - val_loss: 0.9355 - val_accuracy: 0.7651


In [33]:
model_3_pred=model_3.predict(val_sentences)
model_3_pred[:10]

24/24 [==============================] - 1s 6ms/step


array([[3.3233747e-02],
       [7.3633766e-01],
       [9.9998534e-01],
       [6.9961540e-02],
       [7.5713027e-04],
       [9.9991542e-01],
       [7.9406685e-01],
       [9.9999988e-01],
       [9.9999988e-01],
       [6.3346440e-01]], dtype=float32)

In [34]:
model_3_predict=tf.round(tf.squeeze(model_3_pred))
model_3_predict[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

#Bidirectional RNNs

In [36]:
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.Bidirectional(layers.GRU(64, return_sequences=True))(x)
x=layers.Bidirectional(layers.LSTM(64))(x)
outputs=layers.Dense(1,activation="relu")(x)
model_4=tf.keras.Model(inputs,outputs,name="model_4_bidirectional")


In [38]:
model_4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [39]:
history = model_4.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels))

Epoch 1/5
215/215 [==============================] - 19s 54ms/step - loss: 0.1521 - accuracy: 0.9594 - val_loss: 1.8172 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 11s 52ms/step - loss: 0.1426 - accuracy: 0.9695 - val_loss: 2.3265 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 11s 52ms/step - loss: 0.1490 - accuracy: 0.9654 - val_loss: 1.7329 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 10s 47ms/step - loss: 0.1011 - accuracy: 0.9745 - val_loss: 1.7359 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 10s 47ms/step - loss: 0.0902 - accuracy: 0.9793 - val_loss: 2.7508 - val_accuracy: 0.7612


In [40]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 15, 128)          74496     
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                             

In [42]:
model_4_predict=model_4.predict(val_sentences)
model_4_predict[:10]

24/24 [==============================] - 0s 11ms/step


array([[0.        ],
       [0.4723934 ],
       [2.5719943 ],
       [0.03404605],
       [0.        ],
       [2.8084729 ],
       [0.9326427 ],
       [3.4212904 ],
       [3.2136564 ],
       [0.7713453 ]], dtype=float32)

# Using Convolutional layer

In [44]:
embedding_test=embedding(text_vectorizer(["this is a test"]))
conv_1d=layers.Conv1D(filters=10,kernel_size=3#also seen as ngram=5
                      ,activation="relu")
conv_1d_output=conv_1d(embedding_test)
max_pool1d=layers.MaxPool1D(3)
max_pool1D_output=max_pool1d(conv_1d_output)
embedding_test.shape,conv_1d_output.shape, max_pool1D_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 13, 10]), TensorShape([1, 4, 10]))

In [48]:
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.Conv1D(10,5,activation="relu")(x)
x=layers.GlobalMaxPool1D()(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_5=tf.keras.Model(inputs,outputs,name="model_5_conv1d")

In [50]:
model_5.compile(loss= tf.keras.losses.BinaryCrossentropy(),
                 optimizer=tf.keras.optimizers.Adam(),
                 metrics=["accuracy"])

In [52]:
history=model_5.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels))

Epoch 1/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0503 - accuracy: 0.9785 - val_loss: 1.1193 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0474 - accuracy: 0.9781 - val_loss: 1.1717 - val_accuracy: 0.7572
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0467 - accuracy: 0.9803 - val_loss: 1.1736 - val_accuracy: 0.7559
Epoch 4/5
215/215 [==============================] - 5s 21ms/step - loss: 0.0439 - accuracy: 0.9810 - val_loss: 1.1931 - val_accuracy: 0.7467
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0434 - accuracy: 0.9803 - val_loss: 1.1914 - val_accuracy: 0.7598
